In [1]:
import pandas as pd


from zoobot.tensorflow.datasets import rings


Finetuning needs ring images. The "advanced" catalog uses DECaLS images. But the user need not download *every* DECaLS image (100GB).

This notebook identifies which DECaLS galaxies are relevant to ring finetuning and writes their iaunames to a text file. 

I then upload those galaxies (manually) to Dropbox so that users can download them.

(In practice, I use the list to rsync them from the decals pile to a new pile, all on lofar9, then tar.gzip)

In [2]:
df = pd.read_parquet('/Users/walml/repos/zoobot/data/example_ring_catalog_advanced.parquet')

In [3]:
df

,iauname,smooth-or-featured_featured-or-disk_fraction,disk-edge-on_no_fraction,rare-features_ring,rare-features_ring_fraction,rare-features_total,local_png_loc
0,J000000.80+004200.0,0.257492,0.910869,1,0.125000,8,J000/J000000.80+004200.0.png
1,J000001.03+003228.7,0.126510,0.911055,1,0.111111,9,J000/J000001.03+003228.7.png
2,J000001.55-000533.3,0.167366,0.536398,0,0.000000,5,J000/J000001.55-000533.3.png
3,J000001.85+004309.3,0.134337,0.664886,0,0.000000,4,J000/J000001.85+004309.3.png
4,J000002.29-042805.0,0.908152,0.977085,0,0.000000,4,J000/J000002.29-042805.0.png
...,...,...,...,...,...,...,...
253281,J235953.19+001811.8,0.385464,0.972178,0,0.000000,6,J235/J235953.19+001811.8.png
253282,J235953.74+003322.2,0.921103,0.970179,2,0.333333,6,J235/J235953.74+003322.2.png
253283,J235956.86-011324.2,0.300846,0.802763,1,0.500000,2,J235/J235956.86-011324.2.png
253284,J235959.40+083353.0,0.481616,0.974226,0,0.000000,2,J235/J235959.40+083353.0.png


In [4]:
df['file_loc'] = df['local_png_loc'].str.replace('.png', '.jpg', regex=False)
del df['local_png_loc']

In [5]:
# apply selection cuts. must exactly match get_random_ring_catalogs in rings.py

df = rings.apply_selection_cuts(df)

df['label'] = rings.get_rough_class_from_ring_fraction(df['rare-features_ring_fraction'])

In [6]:


df = df.query('label >= 0').reset_index(drop=True)
df['label'].value_counts()  
# these 60k are the images relevant to finetuning

0    50855
1     9947
Name: label, dtype: int64

In [7]:
images_to_copy = df['file_loc']

# [:100]

with open('images_to_copy_decals.txt', 'w') as f:
    f.writelines([x + '\n' for x in images_to_copy])

In [ ]:
rsync -e 'ssh -A -J walml@external.jb.man.ac.uk' /Users/walml/repos/zoobot/zoobot/tensorflow/datasets/images_to_copy_decals.txt walml@lofar9:/nvme1/scratch/walml

rsync -e 'ssh -A -J walml@external.jb.man.ac.uk' /Users/walml/repos/zoobot/zoobot/tensorflow/datasets/images_to_copy_decals.txt walml@galahad.ast.man.ac.uk:/share/nas2/walml

rsync --files-from=/nvme1/scratch/walml/images_to_copy_decals.txt repos/pytorch-galaxy-datasets/roots/decals_dr5/images temp_decals_rings

rsync --files-from=images_to_copy_decals.txt galaxy_zoo/decals/dr5/jpg temp_decals_rings

mv temp_decals_rings advanced

tar -zcvf decals_rings.tar.gz advanced

In [ ]:

rsync -e 'ssh -A -J walml@external.jb.man.ac.uk' walml@galahad.ast.man.ac.uk:/share/nas2/walml/decals_rings.tar.gz /Users/walml/repos/zoobot/data